### Cloud Managing Cloud Storage Access<br>

***

<br>

**Demo**<br>

Google Cloud -> Storage<br>
name: bucket1<br>
region: us-east1-b<br>
storage class: standard<br>
control access: fine-grained<br>
label: key: env, value: test<br>
<br>

Google Cloud -> Compute Engine -> VM Instance -> Create instance<br>
name: instance1<br>
label: key: env, value: test<br>
machine type: e2-micro<br>
access scope: set access for each API: storage: Full<br>
<br>

Google Cloud -> Storage -> bucket1<br>

In [ ]:
gcloud compute ssh instance1 --zone us-east1-b
sudo nano file1.txt
# will open nano editor, you can type in any text
# Ctrl + O to save, enter to verify, Ctrl + X to exit
gsutil cp file1.txt gs://bucket1 # copy the file into the bucket
exit

cd ~
git clone https://github.com/antonitz/google-cloud-associate-cloud-engineer.git
cd google-cloud-associate-cloud-engineer/
cd 12-Storage-Services/01_cloud-storage-management/
gsutil cp *.jpg gs://bucket1


**Making the bucket publicly available**<br>

Bucket -> Permissions -> Add members<br> 
new members: allUsers<br>
role: Storage Object Viewer<br>
<br>

objects -> copy url and paste it in the new tab<br>
back to permissions and remove permissions<br>

click an img -> edit permission -> add entry: public: allUsers<br>
> gsutil acl ch -d AllUsers gs://bucket1/image1.jpg<br>
acl(access control list) ch(change) -d(delete) AllUsers from img1<br>
<br>

**Making signed URL**<br>

Google Cloud -> IAM & Admin -> Service Accounts -> create Service Accounts<br>
account name: signedurl<br>
role: storage object viewer<br>
<br>

select actions -> create key -> json -> download key<br>

Google Cloud -> Cloud Shell -> Shell Editor -> File -> Upload Files -> Upload the key file<br>
rename the file: privatekey.json<br>
Back to the terminal to generate the signed url<br>

In [ ]:
# py-open SSL is needed
pip3 install pyopenssl
gsutil signurl -d 10m privatekey.json gs://bucket1/image1.jpg


you can copy paste the url in the new tab to see the img<br>

**Object Lifecycle Management and Versioning**<br>

Google Cloud -> Cloud Shell<br>

In [ ]:
gsutil versioning get gs://bucket1
# suspended: versioning is not turned on
gsutil versioning set on gs://bucket1 # enables the versioning
gsutil versioning get gs://bucket1 # it'll display enabled now

# delete the file on bucket
gsutil ls -a gs://bucket1 # deleted file still show up

# bring back the nonCurr ver.
gsutil mv gs://bucket1/image1.jpg#NUMBER gs://bucket1/image1.jpg
gsutil ls -a gs://bucket1 # generation number is different

gcloud compute ssh instance1 --zone us-east1-b 
sudo nano file1.txt
# change the text and CTRL + O to save, enter to verify, CTRL + X to exit
gsutil cp file1.txt gs://bucket1
gsutil ls -a gs://bucket1 # you can see 2 versions of file1.txt
gsutil cp gs://bucket1/file1.txt#NUMBER gs://bucket1/file1.txt
gsutil ls -a gs://bucket1 # you'll get more files in diff versions


**add a lifecycle policy**<br>

bucket details -> lifecycle -> add a rule<br>
select delete object<br>
select: days since becoming non-current: 7 days<br>
<br>

add another rule<br>
select set storage class to coldline<br>
select: age: 90 days<br>
<br>


In [ ]:
gsutil lifecycle get gs://bucket1 > lifecycle.JSON # no error = good sign
sudo nano lifecycle.json
# use arrow key to move to the end, and edit 90 days to 120 days
gsutil lifecycle set lifecycle.json gs://bucket1
